In [69]:
import sys
sys.path.insert(0, "/home/aisummer/mikhail_workspace/nlp_service")
import pandas as pd 
import json
import os
import time
import tqdm

from src.DocumentsParser.utils import DBS_DIR_DENSE_VECTORDB_NAME, DBS_DIR_SPARSE_VECTORDB_NAME
from src.evaluation_metrics import ReaderMetrics
from src.DocumentsSummarizer.Summarizer import SummarizerModule
from src.DocumentsSummarizer.utils import SummarizerConfig
from src.utils import DialogueState
from src.logger import Logger
from langchain_core.documents.base import Document

### 1) Создаём объекты

In [70]:
logger = Logger(False)
log = logger.get_logger(__name__)

config = SummarizerConfig.load('/home/aisummer/ssemenov_workspace/nlp-service/config.yaml')
reader = SummarizerModule(config, log)


llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/aisummer/nlp_models/model_llm/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32 

llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.8000 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 8192
llm_load_print_meta: n_embd           = 4096
llm_load_print_meta: n_layer          = 32
llm_load_print_meta: n_head           = 32
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 4
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0e+00
llm_load_print_meta: f_norm_rms_eps   = 1.0

### 2) Указываем инфо для данных

In [71]:
BENCHMARKS_INFO = {'sberquad': 'v1', 'squadv2': 'v1'}
MODEL_PATH = '/home/aisummer/nlp_models/model_llm/blobs/sha256-6a0746a1ec1aef3e7ec53868f220ff6e389f6f8ef87a01d77c96807de94ca2aa'
BENCHES_SIZE = 5
SAVE_LOGFILE = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/logs/trial1.json'
SAVE_CSVFILE_1 = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/csv_log/sberquad_trial1_.csv'
SAVE_CSVFILE_2 = '/home/aisummer/ssemenov_workspace/nlp-service/experiments/reader/benchmark_reader/csv_log/squadv2_trial1.csv'

banchmark_paths = {}
for name, version in BENCHMARKS_INFO.items():
    banchmark_paths[name] = {
        'table': f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/tables/{version}/benchmark.csv",
        'dense_db': f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/dbs/{version}/{DBS_DIR_DENSE_VECTORDB_NAME}",
        'sparse_db':  f"/home/aisummer/ssemenov_workspace/nlp-service/data/{name}/dbs/{version}/{DBS_DIR_SPARSE_VECTORDB_NAME}"
    }

### 3) Загрузка датасета

In [43]:
# загрузить benchmark-датасет
benchmarks_df = {}
for name, bench_path in banchmark_paths.items():
    benchmarks_df[name] = pd.read_csv(banchmark_paths[name]['table'], sep=';')

# инициализировать класс с метриками
base_dir = '/home/aisummer/ssemenov_workspace/nlp-service'
metrics = ReaderMetrics(base_dir)

# logging
logger = Logger(False)
log = logger.get_logger(__name__)

Loading Meteor...
Loading ExactMatch


In [44]:
benchmarks_df['sberquad']

,question,answer,chunk_ids,contexts
0,чем представлены органические остатки?,известковыми выделениями сине-зелёных водорослей,[4974587056],['В протерозойских отложениях органические ост...
1,что найдено в кремнистых сланцах железорудной ...,"нитевидные водоросли, грибные нити",[4974587056],['В протерозойских отложениях органические ост...
2,что встречается в протерозойских отложениях?,органические остатки,[4974587056],['В протерозойских отложениях органические ост...
3,что относится к числу древнейших растительных ...,скопления графито-углистого вещества,[4974587056],['В протерозойских отложениях органические ост...
4,как образовалось графито-углистое вещество?,в результате разложения Corycium enigmaticum,[4974587056],['В протерозойских отложениях органические ост...
...,...,...,...,...
74251,Где используется классическая трёхуровневая си...,в рубиновом лазере.,[1898313107],['Классическая трёхуровневая система накачки р...
74252,какой инвестиционный банк входил в состав Amer...,Lehman Brothers,[4929086505],['Первая платёжная карта American Express появ...
74253,С каким альбомом был созвучен альбом Дэвида Бо...,Low,[1303094225],['Следующий альбом Heroes был во многом созвуч...
74254,В каком техасском оркестре выступал гитарист Л...,Light Crust Doughboys,[934655466],"['Одним из тех, на кого игра Данна произвела н..."


In [45]:
benchmarks_df['squadv2']

,question,answer,chunk_ids
0,When did Beyonce start becoming popular?,in the late 1990s,[621329309]
1,What areas did Beyonce compete in when she was...,singing and dancing,[621329309]
2,When did Beyonce leave Destiny's Child and bec...,2003,[621329309]
3,In what city and state did Beyonce grow up?,"Houston, Texas",[621329309]
4,In which decade did Beyonce become famous?,late 1990s,[621329309]
...,...,...,...
141964,What is the seldom used force unit equal to on...,sthène,[2990931136]
141965,What does not have a metric counterpart?,NaN,[2990931136]
141966,What is the force exerted by standard gravity ...,NaN,[2990931136]
141967,What force leads to a commonly used unit of mass?,NaN,[2990931136]


### 4) Предскажем ответы по вопросам из датасетов 

In [46]:
def get_relevant_ans(df: dict, reader: SummarizerModule, cnt: int, contexts: bool) -> list:
    relevant_ans = []
    questions = []
    context = []
    question = df['question']
    
    for i in tqdm.tqdm(range(cnt)):

        questions.append(question[i])

        if contexts:
            context.append(df['contexts'][i].strip('[]').strip("'"))
            contexts = df['contexts'][i].strip('[]').strip("'")
        else:
            contexts = ''

        relevant_chunks = [contexts.strip('[]').strip("'")]
        state = DialogueState(
            query=question[i],
            base_relevant_docs = [Document(page_content=chunk) for chunk in relevant_chunks])

        
        output = reader.create_answer(state)
        result=''
        for item in output:
            try:
                result += item['choices'][0]['delta']['content']
            except Exception:
                continue
        relevant_ans.append(result)
    return questions, relevant_ans, context

### sberquad 

In [47]:
start_time_sberquad = time.time()

cnt = BENCHES_SIZE
questions_sberquad, predicted_sberquad, context_sberquad = get_relevant_ans(benchmarks_df['sberquad'], reader, cnt, True)
targets_sberquad = benchmarks_df['sberquad']['answer'].to_list()[:cnt]

time_execute_sberquad = time.time() - start_time_sberquad

q_and_a_sberquad = {
    'question': questions_sberquad,
    'predicted_answer': predicted_sberquad,
    'target': targets_sberquad,
    'context': context_sberquad
    }

  0%|          | 0/5 [00:00<?, ?it/s]


llama_print_timings:        load time =    4949.34 ms
llama_print_timings:      sample time =      26.11 ms /    48 runs   (    0.54 ms per token,  1838.66 tokens per second)
llama_print_timings: prompt eval time =    4949.05 ms /   275 tokens (   18.00 ms per token,    55.57 tokens per second)
llama_print_timings:        eval time =    4927.76 ms /    47 runs   (  104.85 ms per token,     9.54 tokens per second)
llama_print_timings:       total time =    9931.73 ms /   322 tokens
 20%|██        | 1/5 [00:09<00:39,  9.94s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    4949.34 ms
llama_print_timings:      sample time =      16.66 ms /    30 runs   (    0.56 ms per token,  1801.26 tokens per second)
llama_print_timings: prompt eval time =     474.74 ms /    26 tokens (   18.26 ms per token,    54.77 tokens per second)
llama_print_timings:        eval time =    3051.70 ms /    29 runs   (  105.23 ms per token,     9.50 tokens per second)
llama_print_timin

In [48]:
df_sberquad = pd.DataFrame(q_and_a_sberquad)
df_sberquad.to_csv(SAVE_CSVFILE_1)
df_sberquad.head(10)

,question,predicted_answer,target,context
0,чем представлены органические остатки?,Органические остатки в протерозойских отложени...,известковыми выделениями сине-зелёных водорослей,В протерозойских отложениях органические остат...
1,что найдено в кремнистых сланцах железорудной ...,"Нитевидные водоросли, грибные нити и формы, бл...","нитевидные водоросли, грибные нити",В протерозойских отложениях органические остат...
2,что встречается в протерозойских отложениях?,В протерозойских отложениях органические остат...,органические остатки,В протерозойских отложениях органические остат...
3,что относится к числу древнейших растительных ...,"Скопления графито-углистого вещества, образова...",скопления графито-углистого вещества,В протерозойских отложениях органические остат...
4,как образовалось графито-углистое вещество?,Образовалось в результате разложения Corycium ...,в результате разложения Corycium enigmaticum,В протерозойских отложениях органические остат...


### squadv2

In [49]:
start_time_squadv2 = time.time()

questions_squadv2, predicted_squadv2, _ = get_relevant_ans(benchmarks_df['squadv2'], reader, cnt, False)
targets_squadv2 = benchmarks_df['squadv2']['answer'].to_list()[:cnt]

time_execute_squadv2 = time.time() - start_time_squadv2

q_and_a_squadv2 = {
    'question': questions_squadv2,
    'predicted_answer': predicted_squadv2,
    'target': targets_squadv2
}

  0%|          | 0/5 [00:00<?, ?it/s]Llama.generate: prefix-match hit



llama_print_timings:        load time =    4949.34 ms
llama_print_timings:      sample time =      52.28 ms /   100 runs   (    0.52 ms per token,  1912.78 tokens per second)
llama_print_timings: prompt eval time =     378.49 ms /    18 tokens (   21.03 ms per token,    47.56 tokens per second)
llama_print_timings:        eval time =   10335.83 ms /    99 runs   (  104.40 ms per token,     9.58 tokens per second)
llama_print_timings:       total time =   10837.97 ms /   117 tokens
 20%|██        | 1/5 [00:10<00:43, 10.84s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    4949.34 ms
llama_print_timings:      sample time =      27.07 ms /    53 runs   (    0.51 ms per token,  1958.25 tokens per second)
llama_print_timings: prompt eval time =     333.86 ms /    18 tokens (   18.55 ms per token,    53.91 tokens per second)
llama_print_timings:        eval time =    5253.16 ms /    52 runs   (  101.02 ms per token,     9.90 tokens per second)
llama_print_timin

In [50]:
df_squadv2 = pd.DataFrame(q_and_a_squadv2)
df_squadv2.to_csv(SAVE_CSVFILE_2)
df_squadv2.head(10)

,question,predicted_answer,target
0,When did Beyonce start becoming popular?,Beyoncé started gaining popularity in the earl...,in the late 1990s
1,What areas did Beyonce compete in when she was...,"When Beyoncé was growing up, she competed in t...",singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,Beyoncé left Destiny's Child in 2000 to pursue...,2003
3,In what city and state did Beyonce grow up?,"Beyoncé grew up in Houston, Texas.","Houston, Texas"
4,In which decade did Beyonce become famous?,The 2000s!,late 1990s


### 6) Посчитаем метрики

In [85]:
benchmarks_score = {}
def metric(predicted: list[str], targets: list[str], name:str) -> None:
    
    length = len(predicted)

    print('Dataset', name, '\n')
    rouge_score = metrics.rouge(predicted, targets[:length])
    print("rouge: ", rouge_score)

    bleu_score = metrics.bleu(predicted, targets[:length])
    print("bleu: ", bleu_score)

    meteor = metrics.meteor(predicted, targets[:length])
    print("meteor: ", meteor)

    exact_match = metrics.exact_match(predicted, targets[:length])
    print("exact_match: ", exact_match, '\n')

    # посчитать метрики 
    score = {
        'rouge': float(rouge_score),
        'bleu': float(bleu_score),
        'meteor': float(meteor),
        'exact_match': float(exact_match)
    }

    return score



score = metric(predicted_sberquad, targets_sberquad,'sberquad')
benchmarks_score['sberquad'] = score

score = metric(predicted_squadv2, targets_squadv2, 'squadv2')
benchmarks_score['squadv2'] = score




Dataset sberquad 

rouge:  0.16
bleu:  0.16303
meteor:  0.73542
exact_match:  0.0 

Dataset squadv2 

rouge:  0.13246
bleu:  0.00432
meteor:  0.23382
exact_match:  0.0 



In [86]:

# сохранить результат
if os.path.exists(SAVE_LOGFILE):
    print("Файл существует!")
    raise ValueError

log_data = {'info': BENCHMARKS_INFO,
            'hyperp': {
                'model_path': MODEL_PATH, 
                'tech_params': config.tech_config.__dict__, 'hyper_params': config.strat_config.__dict__,'benchmark_sizes': BENCHES_SIZE},
                'scores': benchmarks_score}

with open(SAVE_LOGFILE, 'w', encoding='utf-8') as fd:
    fd.write(json.dumps(log_data, indent=1))